# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [70]:
# import libraries
import string
import pandas as pd 
from sqlalchemy import create_engine
import re
import pickle



import nltk
from nltk.tokenize import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score







In [72]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')

query = "SELECT * From DisasterResponse"
df = pd.read_sql_query(query, engine,)

X = df['message'].values
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [73]:
text = "Sentence first. Sentence second. Sentence third"

### 2. Write a tokenization function to process your text data

In [74]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def tokenize(text):
    '''
    Description:
        - Normalization to lowercase
        - Remove punctuation characters
        - Tokenization, lemmatization, and stop word removal
    
    Arguments:
        - text as string
    
    Returns:
        - tokens: a list of tockenized text
    '''
    
    #normalize text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

In [75]:
tokenize(text)

['sentence', 'first', 'sentence', 'second', 'sentence', 'third']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [76]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('forest', MultiOutputClassifier( RandomForestClassifier() ))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [77]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state =123, test_size =0.2)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [66]:
y_train.columns[0]

'related'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [78]:
Y_pred = pipeline.predict(X_test)
Y_pred_df = pd.DataFrame(Y_pred, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('Accuracy: ', accuracy_score(Y_test[column], Y_pred_df[column]))
    print('Feature: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_df[column]))

------------------------------------------------------

Accuracy:  0.8082505729564553
Feature: related

             precision    recall  f1-score   support

          0       0.64      0.49      0.56      1244
          1       0.85      0.91      0.88      3957
          2       0.42      0.23      0.30        35

avg / total       0.80      0.81      0.80      5236

------------------------------------------------------

Accuracy:  0.8810160427807486
Feature: request

             precision    recall  f1-score   support

          0       0.89      0.97      0.93      4334
          1       0.77      0.45      0.56       902

avg / total       0.87      0.88      0.87      5236

------------------------------------------------------

Accuracy:  0.9954163483575248
Feature: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5212
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Feature: food

             precision    recall  f1-score   support

          0       0.95      0.99      0.97      4647
          1       0.85      0.55      0.67       589

avg / total       0.93      0.94      0.93      5236

------------------------------------------------------

Accuracy:  0.9341100076394194
Feature: shelter

             precision    recall  f1-score   support

          0       0.94      0.99      0.97      4795
          1       0.80      0.29      0.43       441

avg / total       0.93      0.93      0.92      5236

------------------------------------------------------

Accuracy:  0.9843391902215431
Feature: clothing

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      5152
          1       0.58      0.08      0.15        84

avg / total       0.98      0.98      0.98      5236

------------------------------------------------------

Accuracy:  0.9761268143621085
Feature: money

             precision    re

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__min_samples_split': [2, 4],
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=4)
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open("model.pickle", 'wb') as pickle_file:
    pickle.dump(cv, pickle_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.